## Based on the code which :
## Iterates through a soundsep csv, opens wav files, saves biosound H5 files for further analysis.
# But instead it just makes short wav files for use in a GAN.

In [1]:
# from scipy import integrate, optimize

import soundfile as sf
import numpy as np
from scipy.signal import spectrogram, butter, sosfilt, sosfiltfilt, correlate
from scipy.io.wavfile import read, write
import IPython.display as ipd
# import sys
from collections import Counter
import os
import pandas as pd
# import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import pickle
import csv
from soundsig.sound import BioSound 
from soundsig.sound import WavFile


# from ipywidgets import interact, fixed
# from ipywidgets import widgets
import matplotlib.pyplot as plt

def plot_a_spectrogram(w, fs:int=48000, title:str=spectrogram, nfft:int=2048):
    ff, tt, Sxx = spectrogram(w, fs=fs, nfft=nfft)
    max_ff = 10000 # only plot the first 10kHz
    ff_index = np.where(ff < max_ff)
    ff = ff[ff_index]
    Sxx = Sxx[ff_index]

    plt.figure(figsize=(10, 5))
    plt.pcolormesh(tt, ff[:max_ff], Sxx[:max_ff], cmap='plasma', shading='auto')
    # plt.pcolormesh(tt, ff[:145], Sxx[:145], cmap='plasma', shading='gouraud')
    plt.title(title)
    plt.xlabel('t (sec)')
    plt.ylabel('Frequency (Hz)')
    plt.grid()


In [2]:
base_folder = '/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/'
#'/Users/williamwood/Documents/Sound_Localization/for_soundsep/2023 maeve and jaewon/14_38_03_18_07_2023-data'
#'/Users/williamwood/Documents/Sound_Localization/for_soundsep/maeves data'#'/Users/williamwood/Documents/Sound_Localization/for_soundsep/14-38-03_18-07-2023' #7_18_23__09_29_44'
wave_files_folder = base_folder
#'/Users/williamwood/Documents/Sound_Localization/for_soundsep/14-38-03_18-07-2023'
# wave_files_folder = '/Users/williamwood/Documents/Sound_Localization/for_soundsep/7_18_23__09_29_44'
# Sound_Localization/for_soundsep/7_18_23__09_29_44'

file_name = os.path.join(base_folder,'_appdata','save','segments.csv')

print(file_name)
df = pd.read_csv(file_name, sep=',',header=0)        
print(f'Length of dataframe: {len(df)}')
print(f'Column names: {df.columns}')
print(f'Tags: {df["Tags"].unique()}')

/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/_appdata/save/segments.csv
Length of dataframe: 4344
Column names: Index(['SourceName', 'SourceChannel', 'StartIndex', 'StopIndex', 'Tags',
       'Coords'],
      dtype='object')
Tags: ['["1"]' '["6"]' '["2"]' '["5"]' '["0"]' '["4"]' '["3"]' '[]']


In [3]:
os.chdir(base_folder)

# This will be the output directory
if not os.path.exists('wavs'):
    os.makedirs('wavs')
    

In [4]:
# Read the csv file as a pandas data frame
soundsepPD = df
nsounds = len(soundsepPD)
print(nsounds)
wav_names = {
    'BroGre1354M':'audio_423212900Hz_14-38-03_18-07-2023.wav',
    'WhiWhi6607F':'audio_432736310Hz_14-38-04_18-07-2023.wav',
    'BluGre1214F':'audio_442360000Hz_14-38-00_18-07-2023.wav',
    'WhiWhi1541M':'audio_458185540Hz_14-37-58_18-07-2023.wav'
}
plotMe=True
normalize = True

# check the wav files to see if they have the same sampling rate
srs = list()
for file in wav_names.values():
    filename = os.path.join(wave_files_folder, file)
    track = sf.SoundFile(filename)
    srs.append(track.samplerate)
    print(f'{filename} has a sample rate of {track.samplerate}')
    track.close()
fs = srs[0]

4344
/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/audio_423212900Hz_14-38-03_18-07-2023.wav has a sample rate of 48000
/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/audio_432736310Hz_14-38-04_18-07-2023.wav has a sample rate of 48000
/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/audio_442360000Hz_14-38-00_18-07-2023.wav has a sample rate of 48000
/Users/williamwood/Documents/Data/Field Station Recordings/for_soundsep/14_38_03_18_07_2023/audio_458185540Hz_14-37-58_18-07-2023.wav has a sample rate of 48000


In [52]:
df

,Unnamed: 0.1,Unnamed: 0,SourceName,SourceChannel,StartIndex,StopIndex,Tags,Coords
0,0,0,BroGre1354M,0,705034,709502,"[""1""]","[8.153053283691406, 11.92204475402832, 5.47700..."
1,1,1,BluGre1214F,2,739121,741816,"[""6""]","[11.00963020324707, 12.903932571411133, 5.4287..."
2,2,2,BluGre1214F,2,773851,776369,"[""2""]","[11.17556381225586, 11.674291610717773, 3.4400..."
3,3,3,BluGre1214F,2,844891,848558,"[""6""]","[10.986668586730957, 12.001487731933594, 3.246..."
4,4,4,BroGre1354M,0,910549,913839,"[""5""]","[9.732298851013184, 11.947017669677734, 0.9106..."
...,...,...,...,...,...,...,...,...
4339,4339,4339,WhiWhi1541M,3,297302736,297305283,"[""5""]",NaN
4340,4340,4340,BluGre1214F,2,297324812,297329004,"[""6""]",NaN
4341,4341,4341,BluGre1214F,2,297367001,297371644,"[""6""]",NaN
4342,4342,4342,BluGre1214F,2,298070022,298074525,"[""6""]",NaN


In [5]:
# def return_voc(df_index):
#     segment = df.iloc[df_index]
#     file_name = wav_names.get(segment['SourceName']) # get the filename from the bird name
#     # print(f'opening data file:{os.path.join(base_folder, wav_names.get(segment['SourceName'] ) ) }')
#     fs, data_wav = read(os.path.join(base_folder, wav_names.get(segment['SourceName'])))
#     chunk = data_wav[segment['StartIndex']:segment['StopIndex']]
#     return chunk


def read_audio_section(filename, start_frame, stop_frame):
    track = sf.SoundFile(filename)

    can_seek = track.seekable() # True
    if not can_seek:
        raise ValueError("Not compatible with seeking")

    sr = track.samplerate
    frames_to_read = stop_frame - start_frame
    track.seek(start_frame)
    audio_section = track.read(frames_to_read)
    track.close()
    return audio_section, sr

In [37]:
# check index below TODO!!!!!
for row_num in range(2):
    go = True
    print(f'row_num={row_num}, startindex={df.iloc[row_num]["StartIndex"]}, stopindex={df.iloc[row_num]["StopIndex"]}')
    dt = df.iloc[row_num]['StopIndex'] - df.iloc[row_num]['StartIndex']
 
    if dt<460: # if the syllable is less than 5ms, skip it 
        print(f'Syllable is too short (dt = {dt}), padding')
        # go = False
        # elif (dt < 460):
        ptadd = int((460 - dt)/2 + 1) # nwinlen requires a syllable of at least 460 samples to calculate biosound stuff, don't know why- Frederic had this at 250 before
        istart = df.iloc[row_num]['StartIndex']-ptadd
        if istart < 0 :
            print ('Skipping short syllable at very start of file')
            go = False
        iend = df.iloc[row_num]['StopIndex'] + ptadd
        # if (iend > len(soundIN.data) ):
        #     print ('Skipping short syllable at eend of file')
        #     go = False
    else:
        istart = df.iloc[row_num]['StartIndex']
        iend = df.iloc[row_num]['StopIndex']
    print(f'go={go} istart={istart} iend={iend} row_num={row_num}')
    # print(f'dt={dt}, syll_start= {int(istart/fs)}, syll_end={int(iend/fs)}')

    if go:
        # soundIN = WavFile(file_name=os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])))
        # soundSYL = soundIN.data[istart:iend]
        # load just the segment of the audio file that we want
        soundSyl, sr = read_audio_section(os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])), start_frame=istart, stop_frame=iend)
        
        birdname = df.iloc[row_num]['SourceName']
        if len(df.iloc[row_num]['Tags']) > 0:
            calltype = df.iloc[row_num]['Tags']
        else:
            calltype = 'unknown'
            
            
        # # Normalize if wanted- I don't want to
        # if normalize :
        #     maxAmp = np.abs(soundIN.data).max() 
        # else:
        #     maxAmp = 1.0
        
        # Create BioSound Object
        # myBioSound = BioSound(soundWave=soundSyl, fs=fs, emitter=birdname, calltype = calltype)

row_num=0, startindex=705034, stopindex=709502
go=True istart=705034 iend=709502 row_num=0
row_num=1, startindex=739121, stopindex=741816
go=True istart=739121 iend=741816 row_num=1


In [27]:
dt

2695

## TODO The cell below reloads one of 4 wav files every loop because they require so much memory. Figure out how to avoid this in the future (ubuntu desktop is probably sufficient)
## but lazy loading wav files makes the most sense. 

In [16]:
def make_wav_file(row_num, plotMe=False):
    go = True
    dt = df.iloc[row_num]['StopIndex'] - df.iloc[row_num]['StartIndex']
    if dt<460: # if the syllable is less than 5ms, try to pad it, and if it's at the start of the file, skip it 
        print(f'Syllable is too short (dt = {dt}), padding')
        # go = False
        # elif (dt < 460):
        ptadd = int((460 - dt)/2 + 1)       
        istart = df.iloc[row_num]['StartIndex']-ptadd
        if istart < 0 :
            print ('Skipping short syllable at very start of file')
            go = False
        iend = df.iloc[row_num]['StopIndex'] + ptadd
    else:
        istart = df.iloc[row_num]['StartIndex']
        iend = df.iloc[row_num]['StopIndex']
    print(f'go={go} istart={istart} iend={iend} row_num={row_num}')
    if go:
        # load just the segment of the audio file that we want
        soundSyl, sr = read_audio_section(os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])), start_frame=istart, stop_frame=iend)
        birdname = df.iloc[row_num]['SourceName']
        if len(df.iloc[row_num]['Tags']) > 0:
            calltype = df.iloc[row_num]['Tags']
        else:
            calltype = 'unknown'
        if normalize :
            maxAmp = np.abs(soundSyl).max() 
        else:
            maxAmp = 1.0

        # Save the wav file
        wav_file_name = os.path.join('wavs', f'{birdname}_{calltype}_{istart}_{iend}.wav')
        write(wav_file_name, fs, soundSyl.astype(np.float32))
        print(f'Wrote {wav_file_name}')
        return wav_file_name

In [7]:
# this shouldn't be necessary in this file
plotMe = False
# normalize = True
# for row_num in range(nsounds):
def make_h5_file(row_num, plotMe=False):
    go = True
    dt = df.iloc[row_num]['StopIndex'] - df.iloc[row_num]['StartIndex']
    if dt<460: # if the syllable is less than 5ms, try to pad it, and if it's at the start of the file, skip it 
        print(f'Syllable is too short (dt = {dt}), padding')
        # go = False
        # elif (dt < 460):
        ptadd = int((460 - dt)/2 + 1) # nwinlen requires a syllable of at least 460 samples to calculate biosound stuff, don't know why- Frederic had this at 250 before
        istart = df.iloc[row_num]['StartIndex']-ptadd
        if istart < 0 :
            print ('Skipping short syllable at very start of file')
            go = False
        iend = df.iloc[row_num]['StopIndex'] + ptadd
        # if (iend > len(soundIN.data) ):
        #     print ('Skipping short syllable at eend of file')
        #     go = False
    else:
        istart = df.iloc[row_num]['StartIndex']
        iend = df.iloc[row_num]['StopIndex']
    print(f'go={go} istart={istart} iend={iend} row_num={row_num}')
    if go:
        # soundIN = WavFile(file_name=os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])))
        # soundSYL = soundIN.data[istart:iend]
        # load just the segment of the audio file that we want
        # soundSyl = read_audio_section(os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])), int(istart/fs), int(iend/fs))
        soundSyl, sr = read_audio_section(os.path.join(wave_files_folder, wav_names.get(df.iloc[row_num]['SourceName'])), start_frame=istart, stop_frame=iend)

        birdname = df.iloc[row_num]['SourceName']
        if len(df.iloc[row_num]['Tags']) > 0:
            calltype = df.iloc[row_num]['Tags']
        else:
            calltype = 'unknown'
            
            
        # # Normalize if wanted- I don't want to
        # if normalize :
        #     maxAmp = np.abs(soundIN.data).max() 
        # else:
        #     maxAmp = 1.0
        
        # Create BioSound Object
        myBioSound = BioSound(soundWave=soundSyl, fs=fs, emitter=birdname, calltype = calltype)
                
        # Calculate the spectrogram and the rms
        myBioSound.spectroCalc(spec_sample_rate=1000, freq_spacing = 100, min_freq=0, max_freq=10000)
        myBioSound.rms = myBioSound.sound.std() 
        
        # Calculate amplitude enveloppe
        myBioSound.ampenv(cutoff_freq = 200, amp_sample_rate = 1000)
        
        # Calculate the power spectrum
        myBioSound.spectrum(f_high=10000)
        
        # Calculate fundamental and related values.  These are the default values.
        # For the estimation of the fundamental, four methods are available: 
        # 'AC' - Peak of the auto-correlation function
        # 'ACA' - Peak of envelope of auto-correlation function 
        # 'Cep' - First peak in cepstrum 
        # 'Stack' - Fitting of harmonic stacks (default - works well for zebra finches)
        
        myBioSound.fundest(maxFund = 6000, minFund = 300, lowFc = 200, highFc = 10000, 
                            minSaliency = 0.5, debugFig = 0, 
                            minFormantFreq = 500, maxFormantBW = 500, windowFormant = 0.1,
                            method='AC')
        
        # Calculate the MPS
        myBioSound.mpsCalc(window=0.01, Norm = True)
                    
        # save the H5 file / biosuoound object
        myBioSound.saveh5(os.path.join(base_folder, 'h5filesAmp', 'bird_%s_call_%s_%s.h5' % (birdname, calltype, row_num)))

        print(f'myBioSound.fund = {myBioSound.fund}')
        if plotMe: 
            print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
            print('                Bird %s    Call Type %s' % (myBioSound.emitter, myBioSound.type))
            myBioSound.plot(DBNOISE=50, f_low=250, f_high=10000)  
            print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [46]:
print('done')

done


In [18]:
# from multiprocessing import Pool

# # run the h5 file creation in parallel

# def test(x):
#     print(x)
# with Pool(4) as p:
#     p.map(test, range(nsounds))


from joblib import Parallel, delayed

res = Parallel(n_jobs=5)(delayed(make_wav_file)(x) for x in range(nsounds))
#make_wav_file(0)

go=True istart=773851 iend=776369 row_num=2
go=True istart=705034 iend=709502 row_num=0
go=True istart=739121 iend=741816 row_num=1
go=True istart=844891 iend=848558 row_num=3
go=True istart=910549 iend=913839 row_num=4
Wrote wavs/BroGre1354M_["1"]_705034_709502.wav
Wrote wavs/BluGre1214F_["6"]_739121_741816.wav
Wrote wavs/BluGre1214F_["2"]_773851_776369.wav
Wrote wavs/BluGre1214F_["6"]_844891_848558.wav
Wrote wavs/BroGre1354M_["5"]_910549_913839.wav
go=True istart=915664 iend=917825 row_num=5
Wrote wavs/BluGre1214F_["6"]_915664_917825.wav
go=True istart=938688 iend=943405 row_num=6
Wrote wavs/BroGre1354M_["1"]_938688_943405.wav
go=True istart=955388 iend=958960 row_num=7
Wrote wavs/BluGre1214F_["6"]_955388_958960.wav
go=True istart=982338 iend=984912 row_num=8
Wrote wavs/BluGre1214F_["0"]_982338_984912.wav
go=True istart=1047560 iend=1050258 row_num=9
Wrote wavs/BluGre1214F_["6"]_1047560_1050258.wav
go=True istart=1169083 iend=1174072 row_num=10
go=True istart=1246128 iend=1249389 row

In [14]:
# Now that the H5 files are saved, we move on..
# except it isn't working.  I'm going to try to do it all in one loop


In [19]:
os.path.join(base_folder, 'h5filesAmp')

'/Users/williamwood/Documents/Data/Field Station Recordings/July_Recordings/14_38_03_18_07_2023_Maeves_data/h5filesAmp'